# Prep Data

In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  
import contractions

In [2]:
%cd datasets

c:\Users\willc\Documents\wills ensemble\datasets


In [3]:
with open('twitter.txt', 'r') as f:
    lines = f.readlines()


In [4]:
twitter_df = pd.DataFrame(columns=["text", "verdict"])
for line in lines:
    sections = line.split('\t')
    verdict = sections[2][:-1]
    if verdict == "non-rumor":
        continue
    if verdict == "unverified":
        verdict = "false"

    data = [sections[1], verdict]
    twitter_df.loc[len(twitter_df)] = data

In [5]:
# download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Clean text

def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]', ' ', text)

def remove_url(text):
    return re.sub('URL', '', text)

def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def clean_text(text):
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords(text)
    text = stemming_words(text)
    text = lemmatize_words(text)
    return text
    
# apply
twitter_df['text'] = twitter_df['text'].apply(clean_text)
twitter_df["text"].values

array(['ohio polic offic hospit eat shard glass sandwich columbu restaur blackliesmatt',
       'one men want copi orlando shooter stop l liber democrat berniebro',
       'missouri governor declar state emerg ahead grand juri', ...,
       'break hostag held insid sydney cafe islam flag held',
       'newsflash orlando shooter muslim democrat republican christian gayprid',
       'great scott backtothefutur almost nail predict hplab make'],
      dtype=object)

In [7]:
print(twitter_df.shape)
twitter_df.to_csv("twitter.csv")

(1729, 2)
